In [ ]:
import nltk
import pickle
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Sentence Tokenizer

In [ ]:
class sentTokenizing:
    def sentTokenize(self,gettingText):
        dataToReSize=[]
        data=[]
        cleanText=''
        for i in gettingText:
            if i=='।' or i=='!' or i=='?':
                cleanText+=i
                dataToReSize.append(''.join(cleanText))
                cleanText=''
            else:
                if i=='\n' or i=='\r' or i=='”' or i=='“' or i=='"':
                    continue
                else:
                    cleanText+=i
        #print (dataToReSize)
        for i in dataToReSize:
            withoutAheadSpace=''
            flag=1
            for j in i:
                if j==' ' and flag:
                    continue
                else:
                    flag=0
                    withoutAheadSpace+=j
            data.append(''.join(withoutAheadSpace))
        #print(data)

        return data

Wordcounter

In [ ]:
class word:
    def sentToWord(self,gettingData):
        cleanSent=''
        for i in gettingData:
            for j in i:
                if j=='”' or j=='“' or j=='"' or j==',' or j=='‘' or j=='’':
                    cleanSent+=' '
                    continue
                elif j=='!' or j=='?' or j=='।':
                    cleanSent+=' '
                    continue
                elif j=='(' or j=='{' or j=='}' or j=='[' or j==']':
                    cleanSent+=' '
                else:
                    if j=='-' or j==':' or j==')':
                        cleanSent+=' '
                    else:
                        cleanSent+=j
        return nltk.word_tokenize(cleanSent)

    def wordCount(self,gettingData):
        b = self.sentToWord(gettingData)
        return len(b)

DL Model

In [ ]:
class SentenceSummarizer:
  def __init__(self, threshold=0.7):
    self.model = SGDClassifier(loss="log_loss", learning_rate="adaptive", eta0=0.01)

  def getweightbywords(self, content_sentences):
    weights=[]
    wc = 0
    for i in content_sentences:
      wc=wc + word().wordCount(i)
    for i in content_sentences:
      wcpl = word().wordCount(i)
      weights.append(wcpl/wc)
    return weights

  def getweightbysimilarity(self, content_sentences):
    weights=[]
    vectorizer = TfidfVectorizer()
    text_vector = vectorizer.fit_transform(content_sentences)

    for i in content_sentences:
      sentence_vector = vectorizer.transform([i])
      weights.append(cosine_similarity(sentence_vector, text_vector)[0][0])
    # print(weights)
    return weights

  def getsummaryvalue(self, content_sentences, summary_sentences):
    s = []
    for i in content_sentences:
        if i in summary_sentences:
          s.append(1)
        else:
          s.append(0)
    return s


  def fit(self, content_sentences, summary_sentences):
    #get weight for every sentences
    w_weights = self.getweightbywords(content_sentences)
    s_weights = self.getweightbysimilarity(content_sentences)
    #get summary_value
    s = self.getsummaryvalue(content_sentences, summary_sentences)
    # Combine features
    X = np.array([[f1, f2] for f1, f2 in zip(w_weights, s_weights)])
    # Use partial_fit to update the model incrementally
    self.model.partial_fit(X, s, classes=[0,1])


  def predict(self, content_sentences):
    #get weight for every sentences
    w_weights = self.getweightbywords(content_sentences)
    s_weights = self.getweightbysimilarity(content_sentences)

    # X = np.array([[w_weights, s_weights]])
    X = np.array([[f1, f2] for f1, f2 in zip(w_weights, s_weights)])
    prediction = self.model.predict(X)
    print("predictions :", prediction)

    p=0
    for i in range(len(prediction)):
      if prediction[i]>=0.7:
        p=p+1

    n=len(content_sentences)

    if n > 3 and p > int(n / 3):
      # Calculate product for each sentence
      raw_predictions = [(1/f1) * f2 for f1, f2 in zip(w_weights, s_weights)]

      # Convert raw_predictions to a NumPy array
      data = [[x] for x in raw_predictions]

      # Scale the predictions using min-max scaling
      scaler = MinMaxScaler(feature_range=(0, 1))
      scaled_data = scaler.fit_transform(data)
      prediction = [x[0] for x in scaled_data]
      print(prediction)


    summary = ""
    for i in range(len(prediction)):
      # print(content_sentences[i])
      if prediction[i]>=0.5:
        summary = summary + content_sentences[i]

    return summary

  def savemodel(self):
      """Saves the trained model to a file."""
      with open("summary_model.pkl", "wb") as f:
          pickle.dump(self, f)

  def loadmodel(self):
      """Loads a pre-trained model from a file."""
      with open("summary_model.pkl", "rb") as f:
          loaded_model = pickle.load(f)
          self.__dict__ = loaded_model.__dict__  # Update current object with loaded attributes



Testing model with simple data

In [ ]:
content_data = [
    "This is the first sentence.",
    "This is the second sentence.",
    "This is the third sentence.",
    "Hello world"
]

summary_data = [
    "This is the first sentence.",
    "This is the third sentence."
]

test_data = [
    "This is the first sentence.",
    "This is the second sentence.",
    "This is the third sentence",
    "This is the fourth sentence",
    "This is the fifth sentence.",
    "Hello world",
    "Heloo Bnagladesh"
]

model = SentenceSummarizer()
model.fit(content_data, summary_data)
generated_summary = model.predict(test_data)
print("Generated Summary:")
# for sentence in generated_summary:
#     print(sentence)
print(generated_summary)

predictions : [1 1 1 1 1 1 1]
[1.0, 0.5380509132588067, 0.6456610959105681, 0.6456610959105681, 0.5380509132588067, 0.0, 0.0]
Generated Summary:
This is the first sentence.This is the second sentence.This is the third sentenceThis is the fourth sentenceThis is the fifth sentence.


Train the model with dataset and saving the model values

In [ ]:
df = pd.read_csv('summary_dataset_1.csv')
model = SentenceSummarizer()
for _,row in df.iterrows():
  try:
    c = sentTokenizing().sentTokenize(row['content'])
    s = sentTokenizing().sentTokenize(row['summary'])
    model.fit(c,s)
  except:
    a = None

test = "মামলা, আদালতে ছোটাছুটির মধ্যেও যুক্তরাষ্ট্রের পেসিডেন্ট পদে রিপাবলিকান পার্টির মনোনয়ন দৌঁড়ে অনেকটাই এগিয়ে ডোনাল্ড ট্রাম্প। সর্বশেষ সাউথ ক্যারোলাইনায় দলের প্রাইমারিতে বড় ব্যবধানে হারিয়েছেন জাতিসংঘে যুক্তরাষ্ট্রের সাবেক রাষ্ট্রদূত নিকি হেইলিকে। নানা অনিশ্চয়তা কাটিয়ে দলীয় মনোনয়নের দৌড়ে ডোনাল্ড ট্রাম্পের বেশ সম্ভাবনা দেখা যাচ্ছে। এর মধ্যেই তিনি নিজের জয়ের ব্যাপারে আত্মবিশ্বাসী হয়ে হুঁশিয়ারি দিয়েছেন প্রেসিডেন্ট জো বাইডেনকে। বাইডেন, ইউ আর ফায়ারড্! গেট আউট, গেট আউট! আসছে নভেম্বরে জো বাইডেনের চোখে চোখ রেখে এই কথাটা বলবেন, ফেব্রুয়ারিতেই সেই ঘোষণা দিয়ে রাখছেন ডোনাল্ড ট্রাম্প। সাউথ ক্যারোলাইনায় দলের প্রাইমারিতে নিকি হেইলির বিরুদ্ধে বড় জয়ের পর রিপাবলিকানদের প্রেসিডেন্ট প্রার্থী হতে এক ধাপ এগিয়ে গেলেন তিনি। দ্বিতীয়বার তার প্রেসিডেন্ট পদে মনোনয়ন পাওয়া নিয়ে ব্যাপক সন্দেহ ছিল খোদ রিপাবলিকান দলের মধ্যেই। কিন্তু এখন মনোনয়নের দৌড়ে তার সাথে রয়েছেন মাত্র একজন প্রার্থী, তাও ট্রাম্প থেকে বেশ খানিকটা পিছিয়ে। সাউথ ক্যারোলাইনা হেইলির নিজের রাজ্য হওয়ায় সাবেক রাষ্ট্রপতির জয়টা বেশি গুরুত্বপূর্ণ। যদিও নিকি হেইলি এখনই লড়াই ছাড়ছেন না। তিনি অন্ততঃ ‘সুপার টুইসডে’ পর্যন্ত প্রতিযোগিতায় থাকার অঙ্গীকার পুনর্ব্যক্ত করেছেন। মার্চের পাঁচ তারিখ সেই মঙ্গলবার। সেদিন ১৬ টি রাজ্যের রিপাবলিকানরা তাদের রায় জানাবেন। সাউথ ক্যারোলাইনার জয় উদযাপন করার সময় মি. ট্রাম্প মিজ হেইলির কথা একবারও উল্লেখ করেননি। তার নজর নভেম্বরের সাধারণ নির্বাচনের দিকে। হোয়াইট হাউসে তারই উত্তরসূরি বাইডেনের সাথে একটি ‘রি-ম্যাচ’ বা পুনঃ লড়াইয়ের সম্ভাবনা এখন প্রবল। শনিবারের ফলাফলের পরে দলের ঐক্যের প্রশংসা করেছেন ট্রাম্প। বলেছেন, এমন মনোভাব আগে কখনও ছিল না। আমি রিপাবলিকান পার্টিকে এতটা ঐক্যবদ্ধ কখনও দেখিনি।"
t = sentTokenizing().sentTokenize(test)
print(t)
generated_summary = model.predict(t)
print("Generated Summary:")
# for sentence in generated_summary:
#     print(sentence)
print(generated_summary)
# model.savemodel()

['মামলা, আদালতে ছোটাছুটির মধ্যেও যুক্তরাষ্ট্রের পেসিডেন্ট পদে রিপাবলিকান পার্টির মনোনয়ন দৌঁড়ে অনেকটাই এগিয়ে ডোনাল্ড ট্রাম্প।', 'সর্বশেষ সাউথ ক্যারোলাইনায় দলের প্রাইমারিতে বড় ব্যবধানে হারিয়েছেন জাতিসংঘে যুক্তরাষ্ট্রের সাবেক রাষ্ট্রদূত নিকি হেইলিকে।', 'নানা অনিশ্চয়তা কাটিয়ে দলীয় মনোনয়নের দৌড়ে ডোনাল্ড ট্রাম্পের বেশ সম্ভাবনা দেখা যাচ্ছে।', 'এর মধ্যেই তিনি নিজের জয়ের ব্যাপারে আত্মবিশ্বাসী হয়ে হুঁশিয়ারি দিয়েছেন প্রেসিডেন্ট জো বাইডেনকে।', 'বাইডেন, ইউ আর ফায়ারড্!', 'গেট আউট, গেট আউট!', 'আসছে নভেম্বরে জো বাইডেনের চোখে চোখ রেখে এই কথাটা বলবেন, ফেব্রুয়ারিতেই সেই ঘোষণা দিয়ে রাখছেন ডোনাল্ড ট্রাম্প।', 'সাউথ ক্যারোলাইনায় দলের প্রাইমারিতে নিকি হেইলির বিরুদ্ধে বড় জয়ের পর রিপাবলিকানদের প্রেসিডেন্ট প্রার্থী হতে এক ধাপ এগিয়ে গেলেন তিনি।', 'দ্বিতীয়বার তার প্রেসিডেন্ট পদে মনোনয়ন পাওয়া নিয়ে ব্যাপক সন্দেহ ছিল খোদ রিপাবলিকান দলের মধ্যেই।', 'কিন্তু এখন মনোনয়নের দৌড়ে তার সাথে রয়েছেন মাত্র একজন প্রার্থী, তাও ট্রাম্প থেকে বেশ খানিকটা পিছিয়ে।', 'সাউথ ক্যারোলাইনা হেইলির নিজের রাজ্য হওয়ায় সাবেক রাষ্ট্রপতির জয়ট

Load saved model and predict summary of new text

In [ ]:
model=SentenceSummarizer()
model.loadmodel()
test = "বেনাপোলের অবকাঠামোগত উন্নয়নসহ আমদানি-রপ্তানি বাণিজ্যের গতি ফেরাতে বেনাপোল-যশোর মহাসড়ক ৬ লেনে উন্নতকরাসহ বাইপাস সড়কের প্রশস্তকরণ, পণ্য লোড আনলোডের সরঞ্জাম এবং জনবল বৃদ্ধির লক্ষে রবিবার বিকেলে সড়ক ও জনপথসহ বিশ্ব ব্যাংকের ১৭ সদস্যের একটি প্রতিনিধিদল বেনাপোল বন্দর ও চেকপোস্ট এলাকা পরিদর্শন করেছেন।এর আগে বেনাপোল বন্দর প্যাসেজ্ঞার টার্মিনালের কনফারেন্স রুমে বন্দর, কাস্টম ও সিএন্ডএফ এজেন্ট ও বন্দর ব্যবহারকারী বিভিন্ন সংগঠনের নেতা ও প্রশাসনের কর্মকর্তাদের সাথে মত বিনিময় করেন প্রতিনিধিদলটি। উন্নয়ন কর্মকাণ্ডের পূর্ব প্রস্ততি হিসেবে প্রতিনিধি দলের বেনাপোল পরিদর্শন বলে জানান বন্দর সংশ্লিষ্টরা।বাংলাদেশ সড়ক ও জনপথের অতিরিক্ত প্রধান প্রকৌশলী রেজা আহম্মেদ জাবের প্রতিনিধিদলের নেতৃত্ব দেন। এ সময় উপস্থিত ছিলেন, বিশ্ব ব্যাংকের প্রতিনিধি মুন্নি জাহান, বাংলাদেশ স্থলবন্দর কর্তৃপক্ষের তত্বাবধায়ক প্রকৌশলী মোহাম্মদ হাসান আলী, স্থলবন্দর বেনাপোল উপপরিচালক মামুন কবির তরফদার, বেনাপোল কাস্টমের সহকারী কমিশনার উত্তম কুমার চাকমা, বন্দরের সহকারী পরিচালক মেহেদী হাসান, বেনাপোল সিএন্ডএফ এজেন্ট এসোসিয়েশন সভাপতি মফিজুর রহমান সজনসহ ভারত, নেপাল, নাইজেরিয়াসহ বিভিন্ন দেশের প্রতিনিধিরা।"
t = sentTokenizing().sentTokenize(test)
print(t)
generated_summary = model.predict(t)
print(generated_summary)



['বেনাপোলের অবকাঠামোগত উন্নয়নসহ আমদানি-রপ্তানি বাণিজ্যের গতি ফেরাতে বেনাপোল-যশোর মহাসড়ক ৬ লেনে উন্নতকরাসহ বাইপাস সড়কের প্রশস্তকরণ, পণ্য লোড আনলোডের সরঞ্জাম এবং জনবল বৃদ্ধির লক্ষে রবিবার বিকেলে সড়ক ও জনপথসহ বিশ্ব ব্যাংকের ১৭ সদস্যের একটি প্রতিনিধিদল বেনাপোল বন্দর ও চেকপোস্ট এলাকা পরিদর্শন করেছেন।', 'এর আগে বেনাপোল বন্দর প্যাসেজ্ঞার টার্মিনালের কনফারেন্স রুমে বন্দর, কাস্টম ও সিএন্ডএফ এজেন্ট ও বন্দর ব্যবহারকারী বিভিন্ন সংগঠনের নেতা ও প্রশাসনের কর্মকর্তাদের সাথে মত বিনিময় করেন প্রতিনিধিদলটি।', 'উন্নয়ন কর্মকাণ্ডের পূর্ব প্রস্ততি হিসেবে প্রতিনিধি দলের বেনাপোল পরিদর্শন বলে জানান বন্দর সংশ্লিষ্টরা।', 'বাংলাদেশ সড়ক ও জনপথের অতিরিক্ত প্রধান প্রকৌশলী রেজা আহম্মেদ জাবের প্রতিনিধিদলের নেতৃত্ব দেন।', 'এ সময় উপস্থিত ছিলেন, বিশ্ব ব্যাংকের প্রতিনিধি মুন্নি জাহান, বাংলাদেশ স্থলবন্দর কর্তৃপক্ষের তত্বাবধায়ক প্রকৌশলী মোহাম্মদ হাসান আলী, স্থলবন্দর বেনাপোল উপপরিচালক মামুন কবির তরফদার, বেনাপোল কাস্টমের সহকারী কমিশনার উত্তম কুমার চাকমা, বন্দরের সহকারী পরিচালক মেহেদী হাসান, বেনাপোল সিএন্ডএফ এজেন্ট এসোসিয়েশন সভা

Loading saved model, predict data from csv file & saving the content summary pair

In [ ]:
model=SentenceSummarizer()
model.loadmodel()
df = pd.read_csv('bensumm_data.csv')
new_df = pd.DataFrame(columns=['content', 'summary', 'bensumm', 'dl'])
for _,row in df.iterrows():
  try:
    text = sentTokenizing().sentTokenize(row['content'])
    summary = model.predict(text)
    new_df = new_df.append({'content': row['content'], 'summary': row['summary'], 'bensumm': row['bensumm'], 'dl': summary}, ignore_index=True)
  except:
    summary = None

new_df.to_csv('final_data.csv')

In [ ]:
df = pd.read_csv('final_data.csv')
print(df['content'][0])
print(df['summary'][0])
print(df['bensumm'][0])
print(df['dl'][0])

ধর্মীয় অনুভূতিতে আঘাতের অভিযোগে করা আরও ১০ মামলায় হাইকোর্ট থেকে জামিন পেয়েছেন সাবেক মন্ত্রী আবদুল লতিফ সিদ্দিকী। বিচারপতি মো. নিজামুল হক ও বিচারপতি মো. ফরিদ আহমদ শিবলীর সমন্বয়ে ঘটিত হাইকোর্ট বেঞ্চ আজ মঙ্গলবার এ আদেশ দেন। আদালতে লতিফ সিদ্দিকীর পক্ষে শুনানি করেন আইনজীবী জ্যোতির্ময় বড়ুয়া। রাষ্ট্রপক্ষে ছিলেন ডেপুটি অ্যাটর্নি জেনারেল শেখ এ কে এম মনিরুজ্জামান। এর আগে গত ২৬ মে ধর্মীয় অনুভূতিতে আঘাতের অভিযোগে করা আরও সাত মামলায় সাবেক এই মন্ত্রীকে ছয় মাসের অন্তর্বর্তী জামিন দিয়েছেন হাইকোর্ট। একই সঙ্গে এসব মামলার কার্যক্রম ছয় মাসের জন্য স্থগিত করেছিলেন আদালত। গত বছর সেপ্টেম্বরে যুক্তরাষ্ট্রের নিউইয়র্কে এক অনুষ্ঠানে হজ ও তাবলিগ জামাত নিয়ে বিরূপ মন্তব্য করে সমালোচনার মুখে পড়েন আবদুল লতিফ সিদ্দিকী। এ ঘটনার পর আওয়ামী লীগের সভাপতিমণ্ডলীর এই সদস্য দল থেকে বহিষ্কৃত হন। একই ঘটনায় ধর্মীয় অনুভূতিতে আঘাত ও কটূক্তির অভিযোগে তাঁর বিরুদ্ধে রাজধানী ঢাকাসহ দেশের বিভিন্ন জেলায় বেশ কয়েকটি মামলা হয়। নির্ধারিত সময়ে আদালতে হাজির না হওয়ায় প্রতিটি মামলায় তাঁর বিরুদ্ধে গ্রেপ্তারি পরোয়ানা জারি করেন আদালত। গত বছরের ২৫ নভ

Result of content from original text
Input csv contains three cloumn
'content', 'summary', 'bensumm'

In [ ]:
df = pd.read_csv('final_data.csv')

sp=[]
bp=[]
dp=[]
k=0
for _,row in df.iterrows():
  s=0
  b=0
  d=0
  try:
    content = sentTokenizing().sentTokenize(row['content'])
    summary = sentTokenizing().sentTokenize(row['summary'])
    bensumm = sentTokenizing().sentTokenize(row['bensumm'])
    dl = sentTokenizing().sentTokenize(row['dl'])
    for i in content:
      if i in summary:
        s=s+1
      if i in bensumm:
        b=b+1
      if i in dl:
        d=d+1
    sp.append(s/len(content))
    bp.append(b/len(content))
    dp.append(d/len(content))
    k=k+1
  except:
    a = None

print("Content from original text :")
print("Human Summary :",sum(sp)*100/k)
print("BENSUMM Summary :",sum(bp)*100/k)
print("DL Summary :",sum(dp)*100/k)

Content from original text :
Human Summary : 3.3256995590469645
BENSUMM Summary : 38.552604399072166
DL Summary : 18.688705163867


In [ ]:
df = pd.read_csv('final_data.csv')

hdl=[]
dhl=[]
k=0
for _,row in df.iterrows():
  hd=0
  dh=0
  try:
    summary = sentTokenizing().sentTokenize(row['summary'])
    dl = sentTokenizing().sentTokenize(row['dl'])
    for i in summary:
      if i in dl:
        hd=hd+1
    for i in dl:
      if i in summary:
        dh=dh+1
    hdl.append(hd/len(content))
    dhl.append(dh/len(content))
    k=k+1
  except:
    a = None

print("Compare with Human Summary:")
print("DL summary similarity with human:",sum(hdl)*100/k)
print("Human summary similarity with DL:",sum(dhl)*100/k)

Compare with Human Summary:
DL summary similarity with human: 0.33670033670033667
Human summary similarity with DL: 0.33670033670033667


In [ ]:
df = pd.read_csv('final_data.csv')

hdl=[]
dhl=[]
k=0
for _,row in df.iterrows():
  hd=0
  dh=0
  try:
    summary = sentTokenizing().sentTokenize(row['bensumm'])
    dl = sentTokenizing().sentTokenize(row['dl'])
    for i in summary:
      if i in dl:
        hd=hd+1
    for i in dl:
      if i in summary:
        dh=dh+1
    hdl.append(hd/len(content))
    dhl.append(dh/len(content))
    k=k+1
  except:
    a = None

print("Compare with BENSUMM Summary:")
print("DL summary similarity with BENSUMM:",sum(hdl)*100/k)
print("BENSUMM summary similarity with DL:",sum(dhl)*100/k)

Compare with BENSUMM Summary:
DL summary similarity with BENSUMM: 8.080808080808078
BENSUMM summary similarity with DL: 8.249158249158247
